In [1]:
from qiskit.circuit.library import RealAmplitudes
from qiskit import Aer, execute

#from pennylane import numpy as np
import numpy as np
import pandas as pd

import qutip

n_wires = 4
n_layers = 2
iteracoes = 3
n_targets = 10


def RealAmplitudes2(n_layers, n_wires, n_targets):

    circuit = RealAmplitudes(n_wires, entanglement='full', reps=n_layers) 


    num_pars = len(circuit.parameters)
    values = np.random.rand(num_pars)
    new_circuit = circuit.assign_parameters(values)


    def normaliza(v):
        v = v / (np.linalg.norm(v))    
        return v

    def cria_alvos(n_targets, n_wires):
        target_states = []
        for i in range(n_targets):
            target_states.append(normaliza(np.random.rand(1, 2**(n_wires))))
        return target_states        


    print("Circuito gerado:\n {}".format(new_circuit.decompose().draw()))

    backend = Aer.get_backend('statevector_simulator')
    ket = execute(new_circuit,backend).result().get_statevector()

    target_states = cria_alvos(n_targets, n_wires)

    #print("Estado gerado pelo modelo: {}".format(ket))

    ket = np.array(ket)


    def ket_in_amplt(v):
        aux = []
        for i in range(len(v)):
            v[i] = np.dot(v[i],v[i])
            v[i] = np.linalg.norm(v[i])
            aux.append(float(v[i].real))
        aux = np.array(aux)    
        return aux
    
    amplt = ket_in_amplt(ket) 


    def fidelity(target_states, amplt):
        fidelity_sum = 0
        for target_state in target_states:
            output_probs = amplt
            fidelity = np.vdot(target_state, output_probs) ** 2
            fidelity_sum += fidelity
    
        average_fidelity = fidelity_sum / len(target_states)
        return average_fidelity        


    def compute_Q_ptrace(ket, N):
        ket = qutip.Qobj(ket, dims=[[2]*(N), [1]*(N)]).unit()
        entanglement_sum = 0
        for k in range(N):
            rho_k_sq = ket.ptrace([k])**2
            entanglement_sum += rho_k_sq.tr()  
   
        Q = 2*(1 - (1/N)*entanglement_sum)
        return Q


    Q = compute_Q_ptrace(ket, n_wires)
    print("\nGrau de Emaranhamento: {}".format(Q))

    E = fidelity(target_states, amplt)
    print("Expressabilidade: {}".format(E))

    #print("\n\nEstado gerado: {}".format(ket))
    #print("\n\n\n\n\n")

    k=[Q,E]
    return k



k=[]

for i in range(iteracoes):
    print("\n=============")
    print("= Modelo: {} =".format(i+1))
    print("=============\n")
    k.append(RealAmplitudes2(n_layers, n_wires, n_targets))


print("\n\nResultados:\n")
for i in range(iteracoes):
    print(" Modelo {} \n Grau de emaranhamento: {} \n Expressabilidade: {} \n".format(i+1,k[i][0],k[i][1]))


def pand(k):
    eman=[]
    exp=[]

    for i in range(iteracoes):
        eman.append(k[i][0])
    for i in range(iteracoes):
        exp.append(k[i][1]) 

    dic = {"Grau de Emaranhamento": eman, "Expressabilidade":exp}
    df = pd.DataFrame(dic)
    return df

df = pand(k)
df.describe()


= Modelo: 1 =

Circuito gerado:
      ┌────────────────────────┐                    ┌───────────────────────┐»
q_0: ┤ Ry(0.0373852539589571) ├──■────■─────────■──┤ Ry(0.438940960180529) ├»
     ├───────────────────────┬┘┌─┴─┐  │         │  └───────────────────────┘»
q_1: ┤ Ry(0.780791336546276) ├─┤ X ├──┼────■────┼──────────────■────────────»
     ├───────────────────────┤ └───┘┌─┴─┐┌─┴─┐  │              │            »
q_2: ┤ Ry(0.202630310917965) ├──────┤ X ├┤ X ├──┼──────────────┼────────────»
     ├───────────────────────┤      └───┘└───┘┌─┴─┐          ┌─┴─┐          »
q_3: ┤ Ry(0.133642118134475) ├────────────────┤ X ├──────────┤ X ├──────────»
     └───────────────────────┘                └───┘          └───┘          »
«                                                                       »
«q_0: ──────────────────────────────────────■──────────────■─────────■──»
«     ┌────────────────────────┐          ┌─┴─┐            │         │  »
«q_1: ┤ Ry(0.0205202378658771) ├──────────

,Grau de Emaranhamento,Expressabilidade
count,3.000000,3.000000
mean,0.153958,0.048574
std,0.013730,0.002072
min,0.144440,0.047185
25%,0.146088,0.047384
50%,0.147736,0.047583
75%,0.158717,0.049269
max,0.169697,0.050955


In [12]:
from qiskit.circuit.library import RealAmplitudes as qRa
from qiskit import QuantumCircuit, Aer, execute
import qiskit

import qutip

from random import uniform
from math import pi as pi
import numpy as np
import pandas as pd
import scipy as sp

import pennylane as qml

n_wires = 4
n_layers = 2
iteracoes = 3



def RealAmplitudes2(n_layers, n_wires):


    circuit = qRa(n_wires, entanglement='full', reps=n_layers) 

    num_pars = len(circuit.parameters)
    values = np.random.rand(num_pars)
    ansatz = circuit.assign_parameters(values)





    #expressibility:

    def random_unitary(N):

        Z = np.random.randn(N, N) + 1.0j * np.random.randn(N, N)
        [Q, R] = sp.linalg.qr(Z)
        D = np.diag(np.diagonal(R) / np.abs(np.diagonal(R)))
        return np.dot(Q, D)


    def haar_integral(num_qubits, samples):

        N = 2**num_qubits
        randunit_density = np.zeros((N, N), dtype=complex)

        
        zero_state = np.zeros(N, dtype=complex)
        zero_state[0] = 1
        
        for _ in range(samples):
            A = np.matmul(zero_state, random_unitary(N)).reshape(-1,1)
            randunit_density += np.kron(A, A.conj().T) 

        randunit_density/=samples

        return randunit_density
        

    def pqc_integral(num_qubits, ansatze, size, samples):

        N = num_qubits
        randunit_density = np.zeros((2**N, 2**N), dtype=complex)

        for _ in range(samples):
            #params = np.random.uniform(-np.pi, np.pi, size)
            ansatz = ansatze
            result = execute(ansatz, backend=Aer.get_backend('statevector_simulator')).result()
            U = np.reshape(result.get_statevector(ansatz, decimals=5),(-1,1))
            randunit_density += np.kron(U, U.conj().T)

        return randunit_density/samples
            



    #entangling:
    def meyer_wallach(circuit, num_qubits, size, sample=1024):
        res = np.zeros(sample, dtype=complex)
        N = num_qubits

        for i in range(sample):
            params = np.random.uniform(-np.pi, np.pi, size)
            # ansatz = circuit.ass(params, N)
            ansatz = circuit
            result = execute(ansatz,backend=Aer.get_backend('statevector_simulator')).result()
            U = result.get_statevector(ansatz, decimals=5)
            entropy = 0
            qb = list(range(N))

            for j in range(N):
                dens = qiskit.quantum_info.partial_trace(U, qb[:j]+qb[j+1:]).data
                trace = np.trace(dens**2)
                entropy += trace

            entropy /= N
            res[i] = 1 - entropy
        
        return 2*np.sum(res).real/sample

    Q = meyer_wallach(ansatz, n_wires, (2,2))   #ultimo parametro ta correto (size)?
    E = np.linalg.norm(haar_integral(n_wires, 2048) - pqc_integral(n_wires, ansatz, (2,2), 2048))

    k=[Q,E]
    return k




k=[]

for i in range(iteracoes):
    print("\n=============")
    print("= Modelo: {} =".format(i+1))
    print("=============\n")
    k.append(RealAmplitudes2(n_layers, n_wires))


print("\n\nResultados:\n")
for i in range(iteracoes):
    print(" Modelo {} \n Grau de emaranhamento: {} \n Expressabilidade: {} \n".format(i+1,k[i][0],k[i][1]))


def pand(k):
    eman=[]
    exp=[]

    for i in range(iteracoes):
        eman.append(k[i][0])
    for i in range(iteracoes):
        exp.append(k[i][1]) 

    dic = {"Grau de Emaranhamento": eman, "Expressabilidade":exp}
    df = pd.DataFrame(dic)
    return df

df = pand(k)
df.describe()



= Modelo: 1 =



KeyboardInterrupt: 

In [10]:
k=RealAmplitudes2(2, 4)

In [11]:
print(k)

[0.7648928187253805, 0.9691162799518549]
